**Load Libraries**



In [ ]:
#Loading required libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
import string
import gensim
import operator
import re

**Load the data**

In [ ]:
# Reading the two datasets
lowes_df = pd.read_csv("lowes.csv")
home_depot_df = pd.read_csv("hd_spec_8.csv")
# Havig a look at top 5 rows of the datasets.
lowes_df.head()
#home_depot_df.head()

,sku,color,channel,department,category,brand,product_id,category_id,department_id,product_name,description,feature_list,style_attributes,pdp_url,feature_image,launch_on,text
0,1000512177,White,Lowes US,Hardware,Curtain Rods,Project Source,fb99a2cbe786dabf37631e567d64a89f511f961ca45522...,[642],[15],Project Source 28-in To 48-in White Steel Sing...,28-in To 48-in White Steel Single Curtain Rod ...,"['Ideal for layered window treatments', 'Mount...","{'Hardware Included': 'Yes', 'Warranty': 'None...",https://www.lowes.com/pd/Project-Source-28-in-...,https://assets.stylumia.com/originals/2022/11/...,2022-11-18,"color is White, brand is Project Source, pro..."
1,1001804078,Gold,Lowes US,Hardware,Curtain Rods,Achim,a3c3790f08e521453526beeda2fe3d5852dc46d7ce87e0...,[642],[15],Achim Innovative Bailey 28-in to 48-in Gold St...,Innovative Bailey 28-in to 48-in Gold Steel Si...,"['Innovative wrap around design', 'All metal r...","{'Hardware Included': 'Yes', 'Warranty': 'None...",https://www.lowes.com/pd/Achim-Innovative-28-i...,https://assets.stylumia.com/originals/2022/11/...,2022-11-18,"color is Gold, brand is Achim, product name ..."
2,1001004130,Antique Brass,Lowes US,Hardware,Curtain Rods,Hart & Harlow,3be5062874474d6dcc8b41fe4d9fcf926fa334f7b26f7f...,[642],[15],Hart & Harlow Hedera 13/16-in 4-sided 28-in To...,Hedera 13/16-in 4-sided 28-in To 48-in Antique...,['Includes four 13/16-in diameter adjustable p...,"{'Hardware Included': 'Yes', 'Color/Finish Fam...",https://www.lowes.com/pd/Hart-Harlow-Hedera-13...,https://assets.stylumia.com/originals/2022/11/...,2022-11-18,"color is Antique Brass, brand is Hart & Harl..."
3,1001006546,Black,Lowes US,Hardware,Curtain Rods,Hart & Harlow,8be40ae7c994e60ae7444e5d7ed2d17498cb73b4610f1d...,[642],[15],Hart & Harlow Cluster 13/16-in Double 28-in to...,Cluster 13/16-in Double 28-in to 48-in Black S...,['Includes four 13/16-in diameter adjustable p...,"{'Hardware Included': 'Yes', 'Color/Finish Fam...",https://www.lowes.com/pd/Hart-Harlow-Cluster-1...,https://assets.stylumia.com/originals/2022/11/...,2022-11-18,"color is Black, brand is Hart & Harlow, prod..."
4,1001003946,Black,Lowes US,Hardware,Curtain Rods,Hart & Harlow,3ff70c3fce0a4af93711f004bb0c7d31f10e200c4cbe6b...,[642],[15],Hart & Harlow Cluster 13/16-in 5-sided 28-in t...,Cluster 13/16-in 5-sided 28-in to 48-in Black ...,['Includes five 13/16-in diameter adjustable p...,"{'Hardware Included': 'Yes', 'Color/Finish Fam...",https://www.lowes.com/pd/Hart-Harlow-Cluster-1...,https://assets.stylumia.com/originals/2022/11/...,2022-11-18,"color is Black, brand is Hart & Harlow, prod..."


In [ ]:
#Selecting the most appropriate texual column to work on in Lowes dataset.
Nlowes= lowes_df["text"]
Nlowes

0      color is  White, brand is  Project Source, pro...
1      color is  Gold, brand is  Achim, product name ...
2      color is  Antique Brass, brand is  Hart & Harl...
3      color is  Black, brand is  Hart & Harlow, prod...
4      color is  Black, brand is  Hart & Harlow, prod...
                             ...                        
392    color is  Antique Brass, brand is  Hart & Harl...
393    color is  Black, brand is  Lumino, product nam...
394    color is  Pewter, brand is  Style Selections, ...
395    color is  Dark Oil Rubbed Bronze, brand is  Lu...
396    color is  Beachwood, brand is  Kenney, product...
Name: text, Length: 397, dtype: object

In [ ]:
#Combining multiple columns in Home Depot dataset to form a new column named 'text' which contains all the relevant data.
home_depot_df["text"] = home_depot_df[['color','style_attributes','feature_list','description','mrp','brand','product_name','spec','lenght','finish','material']].apply(lambda x: " ".join(str(val) for val in x if not pd.isnull(val)), axis=1)
Nhd = home_depot_df["text"]
Nhd

0       Silver {'Finish Family': 'Satin Nickel', 'Rod/...
1       Brown {'Finish Family': 'Brown', 'Rod/Pole Typ...
2       Silver {'Finish Family': 'Silver', 'Rod/Pole T...
3       Silver {'Finish Family': 'Satin Nickel', 'Rod/...
4       Gold {'Finish Family': 'Gold', 'Rod/Pole Type'...
                              ...                        
1188    Vintage Bronze {'Finish Family': 'Bronze', 'Ro...
1189    Brown {'Rod/Pole Type': 'Single', 'Product Wei...
1190    Black {'Finish Family': 'Matte Black', 'Rod/Po...
1191    White {'Finish Family': 'Ivory', 'Rod/Pole Typ...
1192    Brown {'Finish Family': 'Brown', 'Rod/Pole Typ...
Name: text, Length: 1193, dtype: object

**Text preprocessing**

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string


In [ ]:
#Lowercasing the data.
Nlowes = Nlowes.str.lower()
Nhd = Nhd.str.lower()
#Nlowes
Nhd

0       silver {'finish family': 'satin nickel', 'rod/...
1       brown {'finish family': 'brown', 'rod/pole typ...
2       silver {'finish family': 'silver', 'rod/pole t...
3       silver {'finish family': 'satin nickel', 'rod/...
4       gold {'finish family': 'gold', 'rod/pole type'...
                              ...                        
1188    vintage bronze {'finish family': 'bronze', 'ro...
1189    brown {'rod/pole type': 'single', 'product wei...
1190    black {'finish family': 'matte black', 'rod/po...
1191    white {'finish family': 'ivory', 'rod/pole typ...
1192    brown {'finish family': 'brown', 'rod/pole typ...
Name: text, Length: 1193, dtype: object

In [ ]:
#Removing Punctuations in data.
string.punctuation
exclude = string.punctuation
Nlowes = Nlowes.str.translate(str.maketrans('', '', exclude))
Nhd = Nhd.str.translate(str.maketrans('', '', exclude))
#print(Nlowes)
print(Nhd)


0       silver finish family satin nickel rodpole type...
1       brown finish family brown rodpole type single ...
2       silver finish family silver rodpole type singl...
3       silver finish family satin nickel rodpole type...
4       gold finish family gold rodpole type double pr...
                              ...                        
1188    vintage bronze finish family bronze rodpole ty...
1189    brown rodpole type single product weight lb 6 ...
1190    black finish family matte black rodpole type s...
1191    white finish family ivory rodpole type single ...
1192    brown finish family brown rodpole type double ...
Name: text, Length: 1193, dtype: object


In [ ]:
#Removing Stopwords in data
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

a = Nlowes.apply(remove_stopwords)
b = Nhd.apply(remove_stopwords)
#a
b


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


0       silver finish family satin nickel rodpole type...
1       brown finish family brown rodpole type single ...
2       silver finish family silver rodpole type singl...
3       silver finish family satin nickel rodpole type...
4       gold finish family gold rodpole type double pr...
                              ...                        
1188    vintage bronze finish family bronze rodpole ty...
1189    brown rodpole type single product weight lb 6 ...
1190    black finish family matte black rodpole type s...
1191    white finish family ivory rodpole type single ...
1192    brown finish family brown rodpole type double ...
Name: text, Length: 1193, dtype: object

In [ ]:
#Removing duplicates in data.
from collections import Counter

def remove_duplicate(s):
    word_list = s.split(" ")
    word_dict = Counter(word_list)
    result = " ".join(word_dict.keys())
    return result

a_new = a.apply(lambda x: remove_duplicate(x))
b_new = b.apply(lambda x: remove_duplicate(x))
a_new.head()
#b_new.head()

0    color white brand project source product name ...
1    color gold brand achim product name innovative...
2    color antique brass brand hart harlow product ...
3    color black brand hart harlow product name clu...
4    color black brand hart harlow product name clu...
Name: text, dtype: object

In [ ]:
#onverting the strings data to a DataFrame
df_a = a_new.to_frame()
df_b = b_new.to_frame()
df_b

,text
0,silver finish family satin nickel rodpole type...
1,brown finish family rodpole type single produc...
2,silver finish family rodpole type single produ...
3,silver finish family satin nickel rodpole type...
4,gold finish family rodpole type double product...
...,...
1188,vintage bronze finish family rodpole type sing...
1189,brown rodpole type single product weight lb 6 ...
1190,black finish family matte rodpole type single ...
1191,white finish family ivory rodpole type single ...


In [ ]:
#Tokenization and Lemmatization.
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pandas as pd
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

# Tokenize and lemmatize the data row-wise
for index, row in df_a.iterrows():
    text = row['text']  # Replace 'text' with the actual column name containing the text data
    tokens = word_tokenize(text)
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    df_a.at[index, 'lemmatized_text'] = ' '.join(lemmas)  # Create a new column 'lemmatized_text' to store the lemmatized data


for index, row in df_b.iterrows():
    text = row['text']  # Replace 'text' with the actual column name containing the text data
    tokens = word_tokenize(text)
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    df_b.at[index, 'lemmatized_text'] = ' '.join(lemmas)
# Print the updated DataFrame with lemmatized data
print(df_b)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


                                                   text  \
0     silver finish family satin nickel rodpole type...   
1     brown finish family rodpole type single produc...   
2     silver finish family rodpole type single produ...   
3     silver finish family satin nickel rodpole type...   
4     gold finish family rodpole type double product...   
...                                                 ...   
1188  vintage bronze finish family rodpole type sing...   
1189  brown rodpole type single product weight lb 6 ...   
1190  black finish family matte rodpole type single ...   
1191  white finish family ivory rodpole type single ...   
1192  brown finish family rodpole type double produc...   

                                        lemmatized_text  
0     silver finish family satin nickel rodpole type...  
1     brown finish family rodpole type single produc...  
2     silver finish family rodpole type single produ...  
3     silver finish family satin nickel rodpole type...  
4

**Modelling**

In [ ]:
import locale
print(locale.getpreferredencoding())
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
! pip install transformers

UTF-8
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.6 MB/s eta 0:00:00


In [ ]:
#Importing libraries.
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics.pairwise import cosine_similarity
from transformers import TFAutoModel, AutoTokenizer
import tensorflow as tf
from transformers import BertTokenizer




In [21]:
pip install sentence-transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=2cd0ef728e53326f362341f27fac5e4951e4c522cb3d5991e811dae5cb6e808a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [22]:
from sentence_transformers import SentenceTransformer


In [23]:
#Combining both the datasets in a single dataset.
combined_data = df_a['lemmatized_text'].tolist() + df_b['lemmatized_text'].tolist()


In [24]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"  
model = SentenceTransformer(model_name)

In [ ]:
# #Vectorize the data using Transformers
# new_batch_size = 8

# model_name = "sentence-transformers/all-MiniLM-L6-v2"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = TFAutoModel.from_pretrained(model_name)


In [ ]:
# # Step 4: Vectorize the data using Sentence Transformers
# model_name = "sentence-transformers/all-MiniLM-L6-v2"  # Example model, choose the appropriate one for your task
# model = SentenceTransformer(model_name)


In [26]:
# Obtaining the embeddings of the combined data
embeddings = model.encode(combined_data)


In [ ]:
# #Tokenize the combined data
# tokenized_data = tokenizer(combined_data, padding=True, truncation=True, return_tensors="tf")

In [ ]:
# #Vectorize the data using TF-IDF
# vectorizer = TfidfVectorizer()
# combined_vectorized = vectorizer.fit_transform(combined_data)

#Vectorize the data using Bag of Words
#vectorizer = CountVectorizer()
#combined_vectorized = vectorizer.fit_transform(combined_data)

#Normalize the vectorized data for BOW
#combined_vectorized_normalized = normalize(combined_vectorized)

In [27]:
k =5 # Number of nearest neighbors to retrieve
knn = NearestNeighbors(n_neighbors=k)
#knn.fit(combined_vectorized)

#Knn fit for normalized data.(BOW)
#knn.fit(combined_vectorized_normalized)



In [28]:
knn.fit(embeddings)

NearestNeighbors()

In [ ]:


# tokenized_data.batch_size = new_batch_size
# #Obtaining the embeddings of the combined data
# embeddings = model(tokenized_data.input_ids).last_hidden_state

In [ ]:
#Using decision tree
# labels = [0] * len(df_a) + [1] * len(df_b)

# Train the Decision Tree classifier
# clf = DecisionTreeClassifier()
# clf.fit(combined_vectorized, labels)

# Find matches using the trained classifier
# df_a_vectorized = vectorizer.transform(df_a['lemmatized_text'])
# predictions = clf.predict(df_a_vectorized)

# Print matched products and confidence scores
# for i, lowes_product in enumerate(df_a['lemmatized_text']):
#     print("Lowes Product:", lowes_product)
#     print("Matched Home Depot Product:", df_b['lemmatized_text'][predictions[i]])
#     print()

In [ ]:
# #Using TF-IDF
# df_a_vectorized = vectorizer.transform(df_a['lemmatized_text'])
# similarity_scores, indices = knn.kneighbors(df_a_vectorized)

# #Calculating similarity scores using cosine similarity
# # df_a_vectorized = vectorizer.transform(df_a['lemmatized_text'])
# # similarity_scores = cosine_similarity(df_a_vectorized, combined_vectorized)

# #Normalization for BOW
# # lowes_vectorized_normalized = normalize(df_a_vectorized)
# #similarity_scores, indices = knn.kneighbors(lowes_vectorized_normalized)

# #len(similarity_scores)
# #len(indices[0])
# similarity_scores,indices

In [30]:
# Step 6: Find similarities using KNN
lowes_embeddings = model.encode(df_a['lemmatized_text'])
_, indices = knn.kneighbors(lowes_embeddings)

In [ ]:

# for i, product_a in enumerate(df_a['lemmatized_text']):
#     print("Product A:", product_a)

#     print("Matched Products from Dataset B:")
#     for j in range(len(indices[i])):
#         product_b = combined_data[j]
#         confidence_score = 1 - similarity_scores[i][j]  # Calculate confidence score (1 - similarity score)
       
#         print("Product B:",product_b)
#         print("Confidence Score:", confidence_score)
#     print()

In [38]:
for i, lowes_product in enumerate(df_a['lemmatized_text']):
    print("Lowes Product:", lowes_product)
    print("Matched Home Depot Products:")
    for j in indices[i]:
        home_depot_product = combined_data[j]
        cosine_similarity = np.dot(lowes_embeddings[i], embeddings[j]) / (np.linalg.norm(lowes_embeddings[i]) * np.linalg.norm(embeddings[j]))
        confidence_score = (cosine_similarity + 1) / 2  # Normalize the cosine similarity to range [0, 1]
        print(home_depot_product, "Confidence Score:", confidence_score)
    print()

Lowes Product: color white brand project source product name 28in 48in steel single curtain rod description basic economical solution hanging curtain sheers tiered treatment featurelist ideal layered window mounting hardware included easy install 25in projection styleattributes yes warranty none colorfinish family use location indoor wood specie na material manufacturer item number 972007 finial type collection inch 25 maximum length 480 minimum 280 range unspsc 52131700 theme style traditional package quantity 1 diameter 079 ca resident prop 65 warning
Matched Home Depot Products:
color white brand project source product name 28in 48in steel single curtain rod description basic economical solution hanging curtain sheers tiered treatment featurelist ideal layered window mounting hardware included easy install 25in projection styleattributes yes warranty none colorfinish family use location indoor wood specie na material manufacturer item number 972007 finial type collection inch 25 max